In [1]:
from google.colab import drive
drive.mount('/content/drive')
#Import libraries

import numpy as np
import os
from PIL import Image
import shutil

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,UpSampling2D,concatenate
from keras.optimizers import Adagrad,Adam
from keras.callbacks import EarlyStopping,Callback
import cv2 as cv
import sys
from keras.models import load_model
from google.colab.patches import cv2_imshow
from os import listdir
from os.path import isfile, join
import math
import matplotlib.pyplot as plt
from IPython import display

#Move to the right folder
os.chdir('/content/drive/My Drive/MedicalImaging/Generator')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [0]:
base_path= "dataset/images/cropped/"
gt_base_path = "dataset/groundtruth/cropped/"
images = []
gt_images = []
i = 0
print("N.immagini di training: ", len(os.listdir(base_path)))
print("N. groundtruth: ", len(os.listdir(gt_base_path)))
for entry in os.listdir(base_path):
  i+=1
  print("Opening images n.", i)
  image = Image.open(base_path+entry)
  img = cv.imread(base_path+entry,cv.IMREAD_ANYDEPTH)
  img = cv.resize(img,(512,512))
  images.append(img)
  gt_img = cv.imread(gt_base_path+entry[:-3]+'mask.png',cv.IMREAD_ANYDEPTH)
  gt_img = cv.resize(gt_img,(512,512))
  gt_images.append(gt_img/255)
x_train=np.reshape(images,(276,512,512,1))
y_train=np.reshape(gt_images,(276,512,512,1))

In [2]:
model = load_model("GitHubGeneratorAMano.h5")













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
inputs = Input((512,512,1))
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
conv1 = BatchNormalization()(conv1)
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
conv1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
conv2 = BatchNormalization()(conv2)
conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
conv2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
conv3 = BatchNormalization()(conv3)
conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
conv3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
conv4 = BatchNormalization()(conv4)
conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
conv4 = BatchNormalization()(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
conv5 = BatchNormalization()(conv5)
conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
conv5 = BatchNormalization()(conv5)
drop5 = Dropout(0.5)(conv5)
up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
    UpSampling2D(size=(2, 2))(drop5))
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
conv6 = BatchNormalization()(conv6)
conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
conv6 = BatchNormalization()(conv6)
up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
    UpSampling2D(size=(2, 2))(conv6))
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
conv7 = BatchNormalization()(conv7)
conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
conv7 = BatchNormalization()(conv7)
up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
    UpSampling2D(size=(2, 2))(conv7))
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
conv8 = BatchNormalization()(conv8)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
conv8 = BatchNormalization()(conv8)
up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
    UpSampling2D(size=(2, 2))(conv8))
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
conv9 = BatchNormalization()(conv9)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
conv9 = BatchNormalization()(conv9)
conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
conv9 = BatchNormalization()(conv9)
conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
model = Model(inputs=inputs, outputs=conv10)
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
# Early stopping
callbacks = EarlyStopping(
  monitor = 'val_loss',  
  restore_best_weights = True
  )

In [9]:
history=model.fit(x_train, y_train, batch_size=2, epochs=1, verbose=1, validation_split=0.2, shuffle=True)

Train on 220 samples, validate on 56 samples
Epoch 1/1
  8/220 [>.............................] - ETA: 1:27:00 - loss: 0.6605 - acc: 0.8329

KeyboardInterrupt: ignored

In [4]:
base_path= "dataset/images/test_enhanced/"
gt_base_path = "dataset/groundtruth/test_enhanced/"
test_images = []
test_gt_images = []
print(len(os.listdir(base_path)))
print(len(os.listdir(gt_base_path)))
i=0
for entry in os.listdir(base_path):
  i+=1
  print(i)
  image = Image.open(base_path+entry)
  img = cv.imread(base_path+entry,cv.IMREAD_ANYDEPTH)
  img = cv.resize(img,(512,512))
  test_images.append(img)
  gt_img = cv.imread(gt_base_path+entry[:-3]+'mask.png',cv.IMREAD_ANYDEPTH)
  gt_img = cv.resize(gt_img,(512,512))
  test_gt_images.append(gt_img/255)

x_test=np.reshape(test_images,(45,512,512,1))
y_test=np.reshape(test_gt_images,(45,512,512,1))

45
45
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [5]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

45/45 [==============================] - 211s 5s/step
Test loss: 0.7186176247066922
Test accuracy: 0.8312038805749681


In [6]:
import numpy as np
import pandas as pd
pre=model.predict(x_test,verbose=1)
for i in range(len(pre)):
  print("PRE")
  print(np.amin(img))
  print(np.amax(img))
  img = pre[i]*255
  cv2_imshow(img.reshape((512,512)))
  img = y_test[i]*255
  print("TEST")
  print(np.amin(img))
  print(np.amax(img))
  cv2_imshow(img.reshape((512,512)))
  model.save("GitHubGenerator4epochs.h5")

Output hidden; open in https://colab.research.google.com to view.